In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import time
import math

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
labels = train.pop('label')

In [ ]:
labels_onehot = tf.one_hot(labels, depth=10)

In [ ]:
train = train.values

In [ ]:
train = np.array([np.reshape(i, (28,28)) for i in train])

In [ ]:
train = train/255

In [ ]:
train.shape

In [ ]:
plt.imshow(train[9])

In [ ]:
test = test.values
test = np.array([np.reshape(i, (28,28)) for i in test])
test = test/255
test.shape

In [ ]:
plt.imshow(test[8])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, labels, test_size=0.2, random_state = 41)

In [ ]:
y_train = y_train.values

In [ ]:
y_train=y_train.reshape(33600,1)

In [ ]:
y_train.shape

In [ ]:
y_train_onehot = np.zeros((33600,10))
y_train_onehot[np.arange(10), y_train] = 1
y_train_onehot.shape

In [ ]:
X_train = X_train.reshape([33600,28,28,1])
X_train.shape

In [ ]:
img_size = (28, 28)
img_size_flat = (28,28)
img_shape = (28, 28)
num_classes = 10
num_channels = 1

In [ ]:
def plot_images(images, cls_true, cls_pred=None):
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    for i,ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(img_shape), cmap='binary')
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()    

In [ ]:
images = train[0:9]
cls_true = labels[0:9]
plot_images(images=images, cls_true=cls_true)

In [ ]:
def new_conv_net(input, num_input_channels, filter_size, num_filters, stride, window, pool_stride, use_pool=True):
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))
    layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, stride, stride, 1], padding='SAME')
    if use_pool == True:
        layer = tf.nn.max_pool(value=layer, ksize=[1, window, window,1], strides=[1, pool_stride, pool_stride, 1], padding='SAME')
    layer = tf.nn.relu(layer)
    return layer, weights

In [ ]:
def flatten(layer):
    layer = tf.contrib.layers.flatten(layer)
    return layer

In [ ]:
def new_fc(input, num_inputs, num_outputs, use_relu=True):
    weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28 ,1], name='x')
# x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [ ]:
layer_conv1, weight_conv1 = new_conv_net(input=x, num_input_channels=1, filter_size=3, num_filters=16,stride=1, window=2, pool_stride=2)

In [ ]:
layer_conv1

In [ ]:
layer_conv2, weight_conv2 = new_conv_net(input=layer_conv1, num_input_channels=16, filter_size=3, num_filters=36,stride=1, window=2, pool_stride=2, use_pool=False)

In [ ]:
layer_conv2

In [ ]:
layer_conv3, weight_conv3 = new_conv_net(input=layer_conv2, num_input_channels=36, filter_size=3, num_filters=64,stride=1, window=2, pool_stride=2)

In [ ]:
layer_conv3

In [ ]:
layer_flat = tf.contrib.layers.flatten(layer_conv3)

In [ ]:
layer_flat

In [ ]:
num_features = int(layer_flat.shape[1])

In [ ]:
layer_fc1 = new_fc(input=layer_flat,num_inputs=num_features, num_outputs=256, use_relu=True)

In [ ]:
layer_fc1

In [ ]:
layer_fc2 = new_fc(input=layer_fc1, num_inputs=256, num_outputs=10, use_relu=False)

In [ ]:
layer_fc2

In [ ]:
y_pred = tf.nn.softmax(layer_fc2)

In [ ]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

In [ ]:
cost = tf.reduce_mean(cross_entropy)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [ ]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
session = tf.Session()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
train_batch_size = 64

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    
    np.random.seed(seed)            
    m = X.shape[0]                  
    mini_batches = []
        
    
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation]

    
    num_complete_minibatches = math.floor(m/mini_batch_size) 
    for k in range(0, num_complete_minibatches):
        
        mini_batch_X = shuffled_X[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    
    if m % mini_batch_size != 0:
        
        end = m - mini_batch_size * math.floor(m / mini_batch_size)
        mini_batch_X = shuffled_X[:,num_complete_minibatches * mini_batch_size:]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size:]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def optimize(num_iterations):
    total_iterations=0
    start_time = time.time()
    for i in range(num_iterations):
        minibatches = random_mini_batches(X_train, y_train, train_batch_size, 5)
        x_batch, y_batch = minibatches
        feed_dict_train = {x: x_batch, y: y_batch}
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i%100 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i+1, acc))
    total_iterations += num_iterations
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))    

In [ ]:
X_train.shape[1]

In [ ]:
def plot_eg_error(cls_pred, correct):
    incorrect = (correct==False)
    images = data.x_test[incorrect]
    cls_pred = cls_pred[incorrect]
    cls_true = labels[incorrect]
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [ ]:
def plot_confusion_matrix(cls_pred):
    cls_true = labels
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    print(cm)
    plt.matshow(cm)
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
test_batch_size = 64

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    num_test = X_val.shape[0]

    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = X_val[i:j, :]

        labels = y_val[i:j, :]

        feed_dict = {x: images,
                     y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j

    cls_true = y_val

    correct = (cls_true == cls_pred)

    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)


In [ ]:
feed_dict_train = {x: X_train, y_true: y_train_onehot}
session.run(optimizer, feed_dict=feed_dict_train)
if i%100 == 0:
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
    print(msg.format(i+1, acc))